## Post-processing with region-growing

In [4]:
import numpy as np
from utils.util import remove_small_lesions, regiongrow_lesions, get_T
import SimpleITK as sitk
import os, glob, tqdm
from utils.io import read_image, write_image

dir = "D:/data/ContrastCT_IDR/resampled_CCTA"
imdir = "{}/images_resampled".format(dir)
# model = "predictions_GAN_mixCT_adv_cls_MMD_EpochN200_lr5.0e-4wcls_2.0wmmd0.0_DeeplySupervised_NLST_newImp_adam_1012_700"
model = "predictions_GAN_NLST_mixCT_adv_cls_loss_MMD_firstD2.0K_clip0.1_EpochN200_lr5.0e-4_DeeplySupervised_NLST_newImp_adam_1012_700"
max_vol = 500
tdir = "{}/{}_rg_nrs_{}/calcium_masks".format(dir, model, max_vol)
if not os.path.exists(tdir):
    os.makedirs(tdir)
predfiles = glob.glob("{}/{}/calcium_masks/*.mhd".format(dir, model))
print(len(predfiles))
N = len(predfiles)
for i in tqdm.tqdm( range( N ) ):
    predf = predfiles[i]
    case = os.path.split(predf)[-1]
    imf = "{}/{}".format( imdir, case )

    output_filename = "{}/{}".format(tdir, case)
    if os.path.exists(output_filename):
        continue

    # Perform region growing on clacium mask
    image, spacing, origin = read_image(imf)
    mask = read_image(predf, only_data=True)
    imT = get_T( imf )
    mask[image < imT] = 0
    # print(case, imT)

    # Store calcium scores in CSV file
    # mask = remove_small_lesions(mask, spacing)
    try:
        mask = regiongrow_lesions(mask, image, spacing, max_vol=max_vol, Threshold=imT)
    except:
        print('case: ', case, "region growing failed!!")
    # try to remove FP with region growing
    # output_filename = "{}/{}".format(tdir, case)
    write_image(output_filename, mask.astype(np.int16), spacing, origin)

print("Refine with region growing finished")

313
Refine with region growing finished


100%|██████████| 313/313 [6:29:27<00:00, 74.66s/it]    


### Apply heart mask

In [4]:
import numpy as np
from utils.util import remove_small_lesions, regiongrow_lesions, get_T
import SimpleITK as sitk
import os, glob, tqdm
from utils.io import read_image, write_image

dir = "D:/data/ContrastCT_IDR/resampled_CCTA"
imdir = "{}/images_resampled".format(dir)
heartdir = "{}/mask".format(dir)
model = "predictions_GAN_mixCT_adv_cls_MMD_EpochN200_lr5.0e-4wcls_2.0wmmd0.0_DeeplySupervised_NLST_newImp_adam_1012_700_rg_nrs_500"

tdir = "{}/{}_heart/calcium_masks".format(dir, model )
if not os.path.exists(tdir):
    os.makedirs(tdir)
predfiles = glob.glob("{}/{}/calcium_masks/*.mhd".format(dir, model))
print(len(predfiles))
N = len(predfiles)
for i in tqdm.tqdm(range( N ) ):
    predf = predfiles[i]
    case = os.path.split(predf)[-1]

    mask, spacing, origin = read_image( predf )
    heartf = '{}/{}'.format(heartdir, case)
    heart = read_image(heartf, only_data=True)

    # remove FP outside heart
    mask[heart==0] = 0

    output_filename = "{}/{}".format(tdir, case)
    write_image(output_filename, mask.astype(np.int16), spacing, origin)


313


100%|██████████| 313/313 [00:22<00:00, 13.83it/s]


### Apply heart mask and remove small lesion
with volume less than T voxels

In [5]:
import numpy as np
from utils.util import remove_small_lesions, regiongrow_lesions, get_T
import SimpleITK as sitk
import os, glob, tqdm
from utils.io import read_image, write_image

dir = "D:/data/ContrastCT_IDR/resampled_CCTA"
imdir = "{}/images_resampled".format(dir)
heartdir = "{}/mask".format(dir)
# model = "predictions_GAN_mixCT_adv_cls_MMD_EpochN200_lr5.0e-4wcls_2.0wmmd0.0_DeeplySupervised_NLST_newImp_adam_1012_700_rg_nrs_500"
model = "predictions_GAN_NLST_mixCT_adv_cls_loss_MMD_firstD2.0K_clip0.1_EpochN200_lr5.0e-4_DeeplySupervised_NLST_newImp_adam_1012_700_rg_nrs_500"
min_voxel = 2
tdir = "{}/{}_sl_{}_heart/calcium_masks".format(dir, model, min_voxel)
if not os.path.exists(tdir):
    os.makedirs(tdir)
predfiles = glob.glob("{}/{}/calcium_masks/*.mhd".format(dir, model))
print(len(predfiles))
N = len(predfiles)
for i in tqdm.tqdm(range( N ) ):
    predf = predfiles[i]
    case = os.path.split(predf)[-1]

    mask, spacing, origin = read_image( predf )
    heartf = '{}/{}'.format(heartdir, case)
    heart = read_image(heartf, only_data=True)

    # remove FP outside heart
    mask[heart==0] = 0

    # Store calcium scores in CSV file
    mask = remove_small_lesions(mask, spacing, min_voxel)

    output_filename = "{}/{}".format(tdir, case)
    write_image(output_filename, mask.astype(np.int16), spacing, origin)



313


100%|██████████| 313/313 [52:37<00:00, 10.09s/it]  
